# 🐍 Aula Prática – Processamento de Imagem com Python
Notebook pronto para **Google Colab**.

**Como usar:**
1. Rode as células **na ordem** (de cima para baixo).
2. Se algo falhar, **rode novamente a Célula 1 (Setup)**.
3. Explore os blocos visuais (comparações lado a lado, grade de thresholds, sliders, ruído).

In [ ]:
# 🔧 Célula 1 — Setup robusto (clonar, garantir dados/imagem1.png)
import os, shutil, pathlib, subprocess

REPO = "aula-pratica-python"
URL  = "https://github.com/duchio/aula-pratica-python.git"
RAW  = "https://raw.githubusercontent.com/duchio/aula-pratica-python/main/imagem1.png"

# 1) Reset e clone do repo
try:
    # %cd só existe no Jupyter/Colab; se não existir, ignoramos
    get_ipython().run_line_magic("cd", "/content")
except Exception:
    pass

shutil.rmtree(REPO, ignore_errors=True)
print("Clonando:", URL)
subprocess.run(["git", "clone", URL], check=True)

try:
    get_ipython().run_line_magic("cd", f"/content/{REPO}")
except Exception:
    import os
    os.chdir(REPO)

print("Diretório atual:", os.getcwd())

# 2) Garantir que exista dados/imagem1.png
os.makedirs("dados", exist_ok=True)

def baixar_raw():
    try:
        import urllib.request
        urllib.request.urlretrieve(RAW, "dados/imagem1.png")
        return os.path.isfile("dados/imagem1.png")
    except Exception as e:
        print("Falha ao baixar RAW:", e)
        return False

def gerar_sintetica():
    import numpy as np, cv2
    img = np.zeros((240, 240), dtype=np.uint8)
    cv2.circle(img, (120,120), 75, 255, -1)
    cv2.putText(img, "US", (92,220), cv2.FONT_HERSHEY_SIMPLEX, 0.8, 200, 2, cv2.LINE_AA)
    cv2.rectangle(img, (30,30), (210,210), 180, 2)
    cv2.imwrite("dados/imagem1.png", img)

if os.path.isfile("dados/imagem1.png"):
    print("OK: dados/imagem1.png já existe.")
elif os.path.isfile("imagem1.png"):
    print("Movendo imagem da raiz para dados/imagem1.png ...")
    shutil.move("imagem1.png", "dados/imagem1.png")
else:
    print("Tentando baixar a imagem do RAW...")
    if not baixar_raw():
        print("Baixa falhou. Gerando imagem sintética de exemplo...")
        gerar_sintetica()

print("Confirmação final:", os.path.isfile("dados/imagem1.png"))
print("Arquivos em dados/:")
try:
    from pathlib import Path
    print([p.name for p in Path('dados').glob('*')])
except Exception as e:
    print("Não consegui listar dados/:", e)

In [ ]:
# 📦 Célula 2 — Imports e configuração
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Exibir gráficos embutidos
try:
    get_ipython().run_line_magic("matplotlib", "inline")
except Exception:
    pass

# Tentar garantir ipywidgets no Colab/Jupyter
try:
    import ipywidgets as widgets
    from ipywidgets import interact, IntSlider, Dropdown
    # No Colab, ativar o gerenciador de widgets
    try:
        from google.colab import output
        output.enable_custom_widget_manager()
    except Exception:
        pass
except Exception:
    # se não tiver, tenta instalar silenciosamente
    import sys, subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "ipywidgets"], check=False)
    import ipywidgets as widgets
    from ipywidgets import interact, IntSlider, Dropdown

In [ ]:
# 🖼️ Célula 3 — Carregar e exibir imagem original + métricas
img_path = Path('dados') / 'imagem1.png'
if not img_path.exists():
    raise FileNotFoundError(f"Não encontrei {img_path}. Rode a Célula 1 novamente.")

img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
if img is None:
    raise RuntimeError("OpenCV não conseguiu carregar a imagem.")

plt.figure(figsize=(5,5))
plt.imshow(img, cmap='gray')
plt.title('Imagem Original')
plt.axis('off')
plt.show()

print(f"Dimensões: {img.shape} | Média dos pixels: {np.mean(img):.2f}")

In [ ]:
# 🔴 Célula 4 — Sobrepor bordas em vermelho (ultra visual)
def overlay_edges_on_gray(img_gray, t1=100, t2=200, blur_ksize=0):
    base = img_gray.copy()
    if blur_ksize and blur_ksize >= 3 and blur_ksize % 2 == 1:
        base = cv2.GaussianBlur(base, (blur_ksize, blur_ksize), 0)

    edges = cv2.Canny(base, t1, t2)

    # Converter para RGB e pintar borda de vermelho
    rgb = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)
    rgb[edges > 0] = [255, 0, 0]  # vermelho nas bordas

    fig, ax = plt.subplots(1, 2, figsize=(10,5))
    ax[0].imshow(img_gray, cmap='gray'); ax[0].set_title('Imagem Original'); ax[0].axis('off')
    ax[1].imshow(rgb); ax[1].set_title(f'Bordas sobrepostas (t1={t1}, t2={t2}, blur={blur_ksize})'); ax[1].axis('off')
    plt.show()

# Exemplo inicial
overlay_edges_on_gray(img, t1=100, t2=200, blur_ksize=0)

In [ ]:
# 🪟 Célula 5 — Grade de thresholds (com e sem blur)
def grid_thresholds(img_gray, pares=[(50,150),(100,200),(150,250),(200,300)], blur_ksize=0):
    base = img_gray.copy()
    if blur_ksize and blur_ksize % 2 == 1:
        base = cv2.GaussianBlur(base, (blur_ksize, blur_ksize), 0)

    n = len(pares)
    cols = 2
    rows = (n + 1) // 2 if n > 1 else 1
    fig, axes = plt.subplots(rows, cols, figsize=(10, 4*rows))
    axes = axes.ravel() if n > 1 else [axes]

    for ax, (t1, t2) in zip(axes, pares):
        e = cv2.Canny(base, t1, t2)
        ax.imshow(e, cmap='gray')
        ax.set_title(f'Canny t1={t1}, t2={t2}, blur={blur_ksize}')
        ax.axis('off')

    for k in range(len(pares), len(axes)):
        axes[k].axis('off')

    plt.show()

print("Sem blur:")
grid_thresholds(img, blur_ksize=0)
print("Com blur (ksize=7):")
grid_thresholds(img, blur_ksize=7)

In [ ]:
# 🎚️ Célula 6 — Sliders interativos (mexer e ver na hora)
from ipywidgets import interact, IntSlider, Dropdown

@interact(
    t1=IntSlider(value=100, min=0, max=300, step=5, description='t1'),
    t2=IntSlider(value=200, min=0, max=400, step=5, description='t2'),
    blur_ksize=Dropdown(options=[0,3,5,7,9,11], value=0, description='blur k')
)
def _viz(t1, t2, blur_ksize):
    overlay_edges_on_gray(img, t1=t1, t2=t2, blur_ksize=blur_ksize)

In [ ]:
# 🌪️ Célula 7 — Mostrar ruído e por que o blur ajuda
def add_salt_pepper(gray, amount=0.01):
    out = gray.copy()
    num = int(amount * gray.size)
    coords = (np.random.randint(0, gray.shape[0], num), np.random.randint(0, gray.shape[1], num))
    out[coords] = 255
    coords = (np.random.randint(0, gray.shape[0], num), np.random.randint(0, gray.shape[1], num))
    out[coords] = 0
    return out

ruidosa = add_salt_pepper(img, amount=0.02)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(ruidosa, cmap='gray'); ax[0].set_title('Imagem com ruído'); ax[0].axis('off')
ax[1].imshow(cv2.GaussianBlur(ruidosa, (7,7), 0), cmap='gray'); ax[1].set_title('Após blur (7x7)'); ax[1].axis('off')
plt.show()

print("Canny na ruidosa SEM blur:")
overlay_edges_on_gray(ruidosa, 100, 200, blur_ksize=0)
print("Canny na ruidosa COM blur (7):")
overlay_edges_on_gray(ruidosa, 100, 200, blur_ksize=7)

In [ ]:
# 💾 Célula 8 — (Opcional) salvar o último resultado de bordas com thresholds padrão
edges = cv2.Canny(img, 100, 200)
out_path = Path('dados') / 'bordas.png'
cv2.imwrite(str(out_path), edges)
print("Resultado salvo em:", out_path.resolve())